In [1]:
import pandas as pd

In [2]:

df_preventiva = pd.read_excel('PREVENTIVA.xlsx')
df_carreta = pd.read_excel('CARRETA.xlsx')
df_Esl = pd.read_excel('ESL.xlsx')
df_mobile = pd.read_excel('MOBILE.xlsx')
df_bipe = pd.read_excel('BIPE.xlsx')
df_bipe_de_notas = pd.read_excel("BIPE DE NOTAS.xlsx")

In [3]:
df_preventiva.columns

Index(['cidade_cliente', 'PEDIDO 1P/FULL'], dtype='object')

In [7]:
df_carreta.columns

Index(['PEDIDO', 'NF`s', 'ROMANEIO/ID', 'TRANSP / MOTO', 'CD/FILIAL ',
       'DATA_ROMANEIO', 'DATA ENTRADA', 'HORA', 'PREVISÃO ENTREGA', 'TIPO ',
       'PEDIDOS SISTEMA', 'CHAVE', 'OBS', 'UUID'],
      dtype='object')

In [4]:
df_carreta.columns = df_carreta.columns.str.strip()

In [13]:
df_carreta.head(2)

,PEDIDO,NF`s,ROMANEIO/ID,TRANSP / MOTO,CD/FILIAL,DATA_ROMANEIO,DATA ENTRADA,HORA,PREVISÃO ENTREGA,TIPO,PEDIDOS SISTEMA,CHAVE,OBS,UUID
0,1106521713,158581,114206587,7459091,1100,2025-10-31,2025-11-01,01:00:00,2025-11-01,Porta a Porta,NaN,52251047960950110733550070001585811061419842,NaN,c41b6316-569d-44f1-acd2-b5d7be570e63
1,1107281555,159005,114206587,7459091,1100,2025-10-31,2025-11-01,01:00:00,2025-11-01,Porta a Porta,NaN,52251047960950110733550070001590051071263336,NaN,60dd74af-a716-4cd8-b389-407e6a215b83


In [5]:
df_merge = (pd.merge(df_preventiva, df_carreta[['PEDIDO', 'NF`s', 'CHAVE', 'PREVISÃO ENTREGA', 'TIPO']]
                      , left_on='PEDIDO 1P/FULL', right_on='PEDIDO', 
                      how='left').drop(columns=['PEDIDO']))

In [6]:
df_merge.head(2)

,cidade_cliente,PEDIDO 1P/FULL,NF`s,CHAVE,PREVISÃO ENTREGA,TIPO
0,GOIANIA,1123564893,287286.0,52251247960950110733550070002872861081230695,2025-12-18,Porta a Porta
1,GOIANIA,1123599834,287299.0,52251247960950110733550070002872991011896358,2025-12-18,Porta a Porta
